
## Compute the associated average outward flux

To have a good quality gradient of the distance map, perform a light smooth over it. Define convolution kernels in each direction and use them recursively. 

In [1]:
%matplotlib ipympl
import matplotlib.pyplot as plt
from vmtk import vmtkscripts
from vmtk import vtkvmtk
import timeit
import os
import sys
import numpy as np
from vtk.numpy_interface import dataset_adapter as dsa
import random
import networkx as nx
import vtk

from vtkdatawidgets.vtk_binding import VtkJupyterBridge

bridge = VtkJupyterBridge()
sphere = vtk.vtkSphereSource()
bridge.SetInputConnection(sphere.GetOutputPort())
bridge.Update()

imgr = vmtkscripts.vmtkImageReader()
imgr.InputFileName = os.path.join(os.getcwd(), 'test-surf-normal-hole.mha')
imgr.Execute()

mc = vmtkscripts.vmtkMarchingCubes()
mc.Image = imgr.Image
mc.Level = 1000.0
mc.Execute()

In [2]:
import math

In [3]:
surfr = vmtkscripts.vmtkSurfaceReader()
surfr.InputFileName = os.path.join(os.getcwd(), 'test-surf-no-hole.stl')
surfr.Execute()

surf = surfr.Surface

Reading STL surface file.


In [4]:
fedges = vtk.vtkFeatureEdges()
fedges.BoundaryEdgesOn()
fedges.FeatureEdgesOff()
fedges.ManifoldEdgesOff()
fedges.SetInputData(surf)
fedges.Update()

ofedges = fedges.GetOutput()

numEdges = ofedges.GetNumberOfPoints()

In [5]:
if numEdges != 0:
    tempcapper = vmtkscripts.vmtkSurfaceCapper()
    tempcapper.Surface = surf
    tempcapper.Interactive = 0
    tempcapper.Execute()
    
    networkSurface = tempcapper.Surface
else:
    networkSurface = surf

In [6]:
numCells = surf.GetNumberOfCells()
cellToDelete = random.randrange(0, numCells-1)

networkSurface.BuildLinks()
networkSurface.DeleteCell(cellToDelete)
networkSurface.RemoveDeletedCells()

net = vmtkscripts.vmtkNetworkExtraction()
net.Surface = networkSurface
net.AdvancementRatio = 1.001
net.Execute()

network = net.Network

Progress: 100%Progress: 100%


In [7]:
convert = vmtkscripts.vmtkCenterlinesToNumpy()
convert.Centerlines = network
convert.Execute()
ad = convert.ArrayDict

wrapping vtkPolyData object
converting cell data: 
Topology
converting points
converting point data: 
Radius
converting cell connectivity list


In [8]:
cellDataTopology = ad['CellData']['Topology']

In [9]:
nodeIndexToIgnore = np.where(cellDataTopology[:,0] == 0)[0][0]

In [10]:
keepCellConnectivityList = []
pointIdxToKeep = np.array([])
for loopIdx, cellConnectivityList in enumerate(ad['CellData']['CellPointIds']):
    if loopIdx == nodeIndexToIgnore:
        removeCellStartIdx = cellConnectivityList[0]
        removeCellEndIdx = cellConnectivityList[-1]
        removeCellLength = cellConnectivityList.size
        if (removeCellEndIdx + 1) - removeCellStartIdx != removeCellLength:
            raise(ValueError)
        continue
    else:
        rescaledCellConnectivity = np.subtract(cellConnectivityList, removeCellLength, where=cellConnectivityList >= removeCellLength)
        keepCellConnectivityList.append(rescaledCellConnectivity)
        pointIdxToKeep = np.concatenate((pointIdxToKeep, cellConnectivityList)).astype(np.int)
        

In [11]:
newPoints = ad['Points'][pointIdxToKeep]

In [12]:
newRadius = ad['PointData']['Radius'][pointIdxToKeep]

In [13]:
outlist = []
for i, cell in enumerate(keepCellConnectivityList):
    start_time = timeit.default_timer()

    cellStartIdx = cell[0]
    cellEndIdx = cell[-1]
    cellStartPoint = newPoints[cellStartIdx].tolist()
    cellEndPoint = newPoints[cellEndIdx].tolist()
    
    if i == 0:

        cl = vmtkscripts.vmtkCenterlines()
        cl.Surface = networkSurface
        cl.SeedSelectorName = 'pointlist'
        cl.SourcePoints = cellStartPoint
        cl.TargetPoints = cellEndPoint
        cl.Execute()
        
        delaunay = vtk.vtkUnstructuredGrid()
        delaunay.DeepCopy(cl.DelaunayTessellation)
    
    else:
        cl = vmtkscripts.vmtkCenterlines()
        cl.Surface = networkSurface
        cl.DelaunayTessellation = delaunay
        cl.SeedSelectorName = 'pointlist'
        cl.SourcePoints = cellStartPoint
        cl.TargetPoints = cellEndPoint
        cl.Execute()

    cll = vtk.vtkPolyData()
    cll.DeepCopy(cl.Centerlines)
        
    outlist.append(cll)
    elapsed = timeit.default_timer() - start_time
    print(f'loop iteration {i} time: {elapsed}')
    

Cleaning surface.
Triangulating surface.
Computing centerlines.
Computing centerlines...loop iteration 0 time: 30.641220117999183
Cleaning surface.
Triangulating surface.
Computing centerlines.
Computing centerlines...loop iteration 1 time: 5.285391259996686
Cleaning surface.
Triangulating surface.
Computing centerlines.
Computing centerlines...loop iteration 2 time: 5.4299206669966225
Cleaning surface.
Triangulating surface.
Computing centerlines.
Computing centerlines...loop iteration 3 time: 5.247208635999414
Cleaning surface.
Triangulating surface.
Computing centerlines.
Computing centerlines...loop iteration 4 time: 5.607431715994608
Cleaning surface.
Triangulating surface.
Computing centerlines.
Computing centerlines...loop iteration 5 time: 5.802966148999985
Cleaning surface.
Triangulating surface.
Computing centerlines.
Computing centerlines...loop iteration 6 time: 6.137925634000567
Cleaning surface.
Triangulating surface.
Computing centerlines.
Computing centerlines...loop it

In [16]:
new = vtk.vtkAppendPolyData()
for data in outlist:  
    new.AddInputData(data)
new.Update()

combined = new.GetOutput()

In [7]:
cl = vmtkscripts.vmtkCenterlines()
cl.Surface = networkSurface
cl.Execute()

Cleaning surface.
Triangulating surface.
Capping surface.
Please position the mouse and press space to add source points, 'u' to undo
Quit renderer
Please position the mouse and press space to add target points, 'u' to undo
Quit renderer
Computing centerlines.
Computing centerlines...

numberOfCells = surfr.Surface.GetNumberOfCells()
cellToRemove = numberOfCells - 10
cellIds = vtk.vtkIdTypeArray()
cellIds.SetNumberOfComponents(1)
cellIds.InsertNextValue(cellToRemove)

selectionNode = vtk.vtkSelectionNode()
selectionNode.SetFieldType(selectionNode.CELL)
selectionNode.SetContentType(selectionNode.INDICES)
selectionNode.SetSelectionList(cellIds)

selection = vtk.vtkSelection()
selection.AddNode(selectionNode)

extractSelection = vtk.vtkExtractSelection()
extractSelection.SetInputData(0, surf)
extractSelection.SetInputData(1, selection)
extractSelection.Update()
    

select = vmtkscripts.vmtkSurfaceConnectivitySelector()
select.Surface = surfr.Surface
select.Execute()

In [ ]:
conv

In [ ]:
sv = vmtkscripts.vmtkSurfaceViewer()
sv.Surface = surf
sv.Execute()

In [15]:
edit = vmtkscripts.vmtkNetworkEditor()
edit.Network = combined
edit.Execute()

Quit renderer


In [17]:
cv = vmtkscripts.vmtkCenterlineViewer()
cv.Centerlines = combined
cv.PointDataArrayName = 'MaximumInscribedSphereRadius'
cv.Execute()

Quit renderer


In [14]:
convert2 = vmtkscripts.vmtkCenterlinesToNumpy()
convert2.Centerlines = cl.Centerlines
convert2.Execute()
ad2 = convert2.ArrayDict

wrapping vtkPolyData object
converting cell data: 
converting points
converting point data: 
MaximumInscribedSphereRadius
EdgeArray
EdgePCoordArray
converting cell connectivity list


In [15]:
ad2

{'Points': array([[ 32.23905182,  28.26617432,  24.58709526],
        [ 32.19395828,  28.24078369,  24.52036095],
        [ 32.17274475,  28.23486328,  24.50147247],
        [ 32.1423378 ,  28.22855186,  24.45545197],
        [ 32.13501358,  28.22581291,  24.44924736],
        [ 32.04504013,  28.184618  ,  24.40757179],
        [ 31.99899292,  28.16766357,  24.36720467],
        [ 31.99801064,  28.16756058,  24.36677742],
        [ 31.95808029,  28.16340637,  24.3494339 ],
        [ 31.91448021,  28.15681267,  24.31373215],
        [ 31.80249405,  28.11990738,  24.25149155],
        [ 31.80240822,  28.11989975,  24.25141716],
        [ 31.79179764,  28.11895752,  24.24230576],
        [ 31.71370125,  28.09150124,  24.18749619],
        [ 31.65065956,  28.05729294,  24.12779045],
        [ 31.60713005,  28.04289246,  24.09856796],
        [ 31.52129364,  28.02075195,  24.02984619],
        [ 31.50103569,  28.01441574,  24.0176239 ],
        [ 31.40071869,  27.98134041,  23.92095375],
  

In [17]:
rend = vmtkscripts.vmtkRenderer()
rend.Execute()

view = vmtkscripts.vmtkSurfaceViewer()
view.vmtkRenderer = rend.vmtkRenderer
view.Opacity = 0.3
view.Surface = networkSurface
view.Execute()

view2 = vmtkscripts.vmtkCenterlineViewer()
view2.vmtkRenderer = rend.vmtkRenderer
view2.Centerlines = combined
view2.Execute()

Quit renderer
Quit renderer


In [ ]:
write = vmtkscripts.vmtkSurfaceWriter()
write.Surface = net.Network
write.OutputFileName = os.path.join(os.getcwd(), 'test-multi-hole.vtp')
write.Execute()